### Read `.bin` file position data from new sessions and convert to `.pos` file

In [1]:
import os

import numpy as np

In [2]:
def get_header_bstring(file):
    """
    Scan file for the occurrence of 'data_start' and return the header
    as byte string

    Parameters
    ----------
    file (str or path): file to be loaded

    Returns
    -------
    str: header byte content
    """

    header = b''
    with open(file, 'rb') as f:
        for bin_line in f:
            if b'data_start' in bin_line:
                header += b'data_start'
                break
            else:
                header += bin_line
    return header

In [3]:
from BinConverter.core.readBin import get_bin_data, get_raw_pos
from BinConverter.core.CreatePos import create_pos

# Full filename to .bin file
bin_filename = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data/06172021-HPC-B6-RAW/06172021-HPC-B6-RAW.bin'


# Read bin file position data with BinConverter and show first 4 rows
raw_position = get_raw_pos(bin_filename)
print('position data read from `.bin` file with BinConverter tool:\n\n', raw_position[0:4, :].astype(int))


# Save to .pos file
pos_file_from_bin = bin_filename.replace('.bin', '.pos')
create_pos(pos_file_from_bin, raw_position)


# Read .pos file data and show first 4 rows
bytes_packet = 20
footer_size = len('\r\ndata_end\r\n')
header_size = len(get_header_bstring(pos_file_from_bin))
num_bytes = os.path.getsize(pos_file_from_bin) - header_size - footer_size
num_packets = num_bytes // bytes_packet

# Set dtypes
pos_dt = np.dtype([('t', ">i4"), ('X', ">i2"), ('Y', ">i2"), ('x', ">i2"), ('y', ">i2"), 
                   ('PX', ">i2"), ('px', ">i2"), ('tot_px', ">i2"), ('unused', ">i2")])

# Read position data from .pos file after conversion from .bin
np_pos = np.memmap(
    filename=pos_file_from_bin,
    dtype=pos_dt,
    mode='r',
    offset=len(get_header_bstring(pos_file_from_bin)),
    shape=(num_packets, ),
)
print('\n\nposition data read from `.pos` file with numpy (after converting from .bin with BinConverter):\n\n', np_pos[0:4,])


position data read from `.bin` file with BinConverter tool:

 [[150121    137     42   1023   1023      0     17      0     17]
 [150122    137     44   1023   1023      0     10      0     10]
 [150123    137     42   1023   1023      0     13      0     13]
 [150124    136     44   1023   1023      0     15      0     15]]


position data read from `.pos` file with numpy (after converting from .bin with BinConverter):

 [(150121, 137, 42, 1023, 1023, 0, 17, 0, 17)
 (150122, 137, 44, 1023, 1023, 0, 10, 0, 10)
 (150123, 137, 42, 1023, 1023, 0, 13, 0, 13)
 (150124, 136, 44, 1023, 1023, 0, 15, 0, 15)]


When reading the `.bin` data using custom code (with numpy), the data is sampled at 100 Hz and contains additional samples in the beginning compared to when reading the `.bin` data with `BinConverter` (which is at 50 Hz).

In [52]:
pos_dt_se = np.dtype([('t', "<i4"), ('X', "<i2"), ('Y', "<i2"), ('x', "<i2"), ('y', "<i2"), 
                   ('PX', "<i2"), ('px', "<i2"), ('tot_px', "<i2"), ('unused', "<i2")])

bin_dt = np.dtype([('id', "S4"), ('packet', "<i4"), ('di', "<i2"), ('si', "<i2"),
                   ('pos', pos_dt_se),
                   ('ephys', np.byte, 384),
                   ('trailer', np.byte, 16)
])

np_bin = np.memmap(
    filename=bin_filename,
    dtype=bin_dt,
    mode='r',
    offset=0,
)

pos_mask = [np_bin['id'] == b'ADU2']

pos_data = np_bin['pos'][pos_mask]

print('Reading .bin position data with numpy:\n\n', pos_data[0:14, ])

print('\n\nposition data read from `.pos` file with numpy (after converting from .bin with BinConverter):\n\n', np_pos[0:4,])

Reading .bin position data with numpy:

 [(125330, 56, 152, 1023, 1023, 0, 16, 0, 16)
 (150120, 44, 136, 1023, 1023, 0, 11, 0, 11)
 (150120, 44, 136, 1023, 1023, 0, 11, 0, 11)
 (150121, 42, 137, 1023, 1023, 0, 17, 0, 17)
 (150121, 42, 137, 1023, 1023, 0, 17, 0, 17)
 (150122, 44, 137, 1023, 1023, 0, 10, 0, 10)
 (150122, 44, 137, 1023, 1023, 0, 10, 0, 10)
 (150123, 42, 137, 1023, 1023, 0, 13, 0, 13)
 (150123, 42, 137, 1023, 1023, 0, 13, 0, 13)
 (150124, 44, 136, 1023, 1023, 0, 15, 0, 15)
 (150124, 44, 136, 1023, 1023, 0, 15, 0, 15)
 (150125, 43, 137, 1023, 1023, 0, 15, 0, 15)
 (150125, 43, 137, 1023, 1023, 0, 15, 0, 15)
 (150126, 45, 138, 1023, 1023, 0, 16, 0, 16)]


position data read from `.pos` file with numpy (after converting from .bin with BinConverter):

 [(150121, 137, 42, 1023, 1023, 0, 17, 0, 17)
 (150122, 137, 44, 1023, 1023, 0, 10, 0, 10)
 (150123, 137, 42, 1023, 1023, 0, 13, 0, 13)
 (150124, 136, 44, 1023, 1023, 0, 15, 0, 15)]


/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/numpy/core/memmap.py:334: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)


Where does this discrepancy come from?

Every second sample is taken to reach a sampling rate of 50 Hz. But why are the first few samples missing?

In [5]:
from nwb_conversion_tools.datainterfaces.ecephys.axona.axonadatainterface import (
    parse_generic_header, establish_mmap_to_position_data, read_bin_file_position_data
)
import mmap
import contextlib

In [6]:
pos_bin_conversion_tools = read_bin_file_position_data(bin_filename)
pos_bin_conversion_tools.astype(int)[0:14, :]

array([[   0,  152, 1023, 1023,    0,   16,    0,   16],
       [   0,  136, 1023, 1023,    0,   11,    0,   11],
       [   3,  136, 1023, 1023,    0,   11,    0,   11],
       [  13,  137, 1023, 1023,    0,   17,    0,   17],
       [  23,  137, 1023, 1023,    0,   17,    0,   17],
       [  33,  137, 1023, 1023,    0,   10,    0,   10],
       [  43,  137, 1023, 1023,    0,   10,    0,   10],
       [  53,  137, 1023, 1023,    0,   13,    0,   13],
       [  63,  137, 1023, 1023,    0,   13,    0,   13],
       [  73,  136, 1023, 1023,    0,   15,    0,   15],
       [  83,  136, 1023, 1023,    0,   15,    0,   15],
       [  93,  137, 1023, 1023,    0,   15,    0,   15],
       [ 103,  137, 1023, 1023,    0,   15,    0,   15],
       [ 113,  138, 1023, 1023,    0,   16,    0,   16]])

It does not happen in my code, so let's go through the original code from `BinConverter` step by step:

In [7]:
from BinConverter.core.readBin import find_n

byte_count = os.path.getsize(bin_filename)
bytes_per_iteration = 432
iteration_count = int(byte_count / bytes_per_iteration)
# sample_count = iteration_count * 192  # each iteration has 192 samples (64*3)

optimal_iteration = 1000000
if optimal_iteration >= iteration_count:
    simul_iterations = iteration_count
else:
    simul_iterations = find_n(iteration_count, optimal=optimal_iteration)

In [8]:
n = int(iteration_count / simul_iterations)  # finds how many loops to do
byte_chunksize = int(simul_iterations * bytes_per_iteration)

DaqFs = 48000
duration = iteration_count * 3 / DaqFs
duration = np.ceil(duration)

pos_Fs = 50
n_samples = int(duration * pos_Fs)

In [9]:
n_samples

15050

In [19]:
raw_pos = np.array([]).astype(float)
mode = 'mmap'

with open(bin_filename, 'rb') as f:

    if mode == 'mmap':
        # we will use memory mapped objects

        with contextlib.closing(mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)) as m:
            
            a = len(m) / bytes_per_iteration
            
            num_iterations = int(len(m) / bytes_per_iteration)

            byte_ids = np.ndarray((num_iterations,), 'S4', m, 0, bytes_per_iteration)

            pos_bool = np.where(byte_ids == b'ADU2')  # valid position packets have an ADU2 header b'ADU2' in bytes

            valid_iterations = np.arange(len(byte_ids))[pos_bool]  # getting the packet number

            # timestamp starts at the 12th bit from the start
            time_stamp = np.ndarray((num_iterations,), np.int32, m, 12, bytes_per_iteration)[pos_bool].reshape(
                (-1, 1))  # gettinge the time stamp data

            # position values start from the 16th bit
            raw_pos = np.ndarray((num_iterations,), (np.int16, (1, 8)), m, 16,
                                 (bytes_per_iteration,)).reshape((-1, 8))[pos_bool][:]

            raw_pos = np.hstack((valid_iterations.reshape((-1, 1)), time_stamp, raw_pos)).astype(
                float)  # stacking all these values to create one matrix

In [47]:
raw_pos_step1 = raw_pos
raw_pos_step1[:, 2:6] = raw_pos[:, [3, 2, 5, 4]]

# find the first valid sample, since the data is sampled at 48kHz, and there are 3 samples per packet, the packet
# rate is 16kHz. The positions are sampled at 50 Hz thus there is a valid position ever 320 packets. The valid position
# will essentially take the last ADU2 headered packet values

first_sample_index = len(
    np.where(raw_pos[:, 0] <= 320 - 1)[0]) - 1  # subtract one since indices in python start at 0

raw_pos_step2 = raw_pos_step1[first_sample_index:, :]

# indices = np.arange(1,raw_pos.shape[0], 2)
indices = np.arange(0, raw_pos_step2.shape[0], 2)
if indices[-1] >= raw_pos_step2.shape[0]:
    indices = indices[:-1]

# raw_pos[indices, 2:4] = 1023
raw_pos_step3 = raw_pos_step2[indices, :]

In [51]:
import pandas as pd

print('pos data (BinConverter) step 1 (read all data):\n')
print(pd.DataFrame(
    raw_pos_step1[0:11, :].astype(int),
    columns=['Index', 'timestamp', 'X', 'Y', 'x', 'y', 'PX', 'px', 'PX+px', 'unused']
))

print('\n\npos data (BinConverter) step 2 (select only the last sample within the first 320 observations):\n')
print(pd.DataFrame(
    raw_pos_step2[0:8, :].astype(int),
    columns=['Index', 'timestamp', 'X', 'Y', 'x', 'y', 'PX', 'px', 'PX+px', 'unused']
))

print('\n\npos data (BinConverter) step 3 (select every second sample for 50 Hz resolution)')
print('Note that no filter is used, so the point of having a 100 Hz sampling rate to avoid aliasing is mute:\n')
print(pd.DataFrame(
    raw_pos_step3[0:4, :].astype(int),
    columns=['Index', 'timestamp', 'X', 'Y', 'x', 'y', 'PX', 'px', 'PX+px', 'unused']
))

pos data (BinConverter) step 1 (read all data):

    Index  timestamp   X    Y     x     y  PX  px  PX+px  unused
0       1     125330  56  152  1023  1023   0  16      0      16
1      14     150120  44  136  1023  1023   0  11      0      11
2      56     150120  44  136  1023  1023   0  11      0      11
3     216     150121  42  137  1023  1023   0  17      0      17
4     376     150121  42  137  1023  1023   0  17      0      17
5     536     150122  44  137  1023  1023   0  10      0      10
6     696     150122  44  137  1023  1023   0  10      0      10
7     856     150123  42  137  1023  1023   0  13      0      13
8    1016     150123  42  137  1023  1023   0  13      0      13
9    1176     150124  44  136  1023  1023   0  15      0      15
10   1336     150124  44  136  1023  1023   0  15      0      15


pos data (BinConverter) step 2 (select only the last sample within the first 320 observations):

   Index  timestamp   X    Y     x     y  PX  px  PX+px  unused
0    216

In [34]:
import struct

def getpos(pos_fpath, arena, method='', flip_y=True):
    """
    getpos function:
    ---------------------------------------------
    variables:
    -pos_fpath: the full path (C:\example\session.pos)
    output:
    t: column numpy array of the time stamps
    x: a column array of the x-values (in pixels)
    y: a column array of the y-values (in pixels)
    """

    with open(pos_fpath, 'rb+') as f:  # opening the .pos file
        headers = ''  # initializing the header string
        for line in f:  # reads line by line to read the header of the file
            # print(line)
            if 'data_start' in str(line):  # if it reads data_start that means the header has ended
                headers += 'data_start'
                break  # break out of for loop once header has finished
            elif 'duration' in str(line):
                headers += line.decode(encoding='UTF-8')
            elif 'num_pos_samples' in str(line):
                num_pos_samples = int(line.decode(encoding='UTF-8')[len('num_pos_samples '):])
                headers += line.decode(encoding='UTF-8')
            elif 'bytes_per_timestamp' in str(line):
                bytes_per_timestamp = int(line.decode(encoding='UTF-8')[len('bytes_per_timestamp '):])
                headers += line.decode(encoding='UTF-8')
            elif 'bytes_per_coord' in str(line):
                bytes_per_coord = int(line.decode(encoding='UTF-8')[len('bytes_per_coord '):])
                headers += line.decode(encoding='UTF-8')
            elif 'timebase' in str(line):
                timebase = (line.decode(encoding='UTF-8')[len('timebase '):]).split(' ')[0]
                headers += line.decode(encoding='UTF-8')
            elif 'pixels_per_metre' in str(line):
                ppm = float(line.decode(encoding='UTF-8')[len('pixels_per_metre '):])
                headers += line.decode(encoding='UTF-8')
            elif 'min_x' in str(line) and 'window' not in str(line):
                min_x = int(line.decode(encoding='UTF-8')[len('min_x '):])
                headers += line.decode(encoding='UTF-8')
            elif 'max_x' in str(line) and 'window' not in str(line):
                max_x = int(line.decode(encoding='UTF-8')[len('max_x '):])
                headers += line.decode(encoding='UTF-8')
            elif 'min_y' in str(line) and 'window' not in str(line):
                min_y = int(line.decode(encoding='UTF-8')[len('min_y '):])
                headers += line.decode(encoding='UTF-8')
            elif 'max_y' in str(line) and 'window' not in str(line):
                max_y = int(line.decode(encoding='UTF-8')[len('max_y '):])
                headers += line.decode(encoding='UTF-8')
            elif 'pos_format' in str(line):
                headers += line.decode(encoding='UTF-8')
                if 't,x1,y1,x2,y2,numpix1,numpix2' in str(line):
                    two_spot = True
                else:
                    two_spot = False
                    print('The position format is unrecognized!')

            elif 'sample_rate' in str(line):
                sample_rate = float(line.decode(encoding='UTF-8').split(' ')[1])
                headers += line.decode(encoding='UTF-8')

            else:
                headers += line.decode(encoding='UTF-8')

    if two_spot:
        '''Run when two spot mode is on, (one_spot has the same format so it will also run here)'''
        with open(pos_fpath, 'rb+') as f:
            '''get_pos for one_spot'''
            pos_data = f.read()  # all the position data values (including header)
            pos_data = pos_data[len(headers):-12]  # removes the header values

            byte_string = 'i8h'

            pos_data = np.asarray(struct.unpack('>%s' % (num_pos_samples * byte_string), pos_data))
            pos_data = pos_data.astype(float).reshape((num_pos_samples, 9))
    return pos_data

In [38]:
print('\n\n\nposition data read from `.pos` file with numpy (after converting from .bin with BinConverter):\n\n', np_pos[0:4,])

getpos(pos_file_from_bin, arena='BehaviorRoom').astype(int)[0:4, :]




position data read from `.pos` file with numpy (after converting from .bin with BinConverter):

 [(2357, 55, 74, 1023, 1023, 0, 17, 0, 17)
 (2358, 56, 76, 1023, 1023, 0, 13, 0, 13)
 (2359, 57, 74, 1023, 1023, 0, 13, 0, 13)
 (2360, 54, 78, 1023, 1023, 0, 12, 0, 12)]


array([[2357,   55,   74, 1023, 1023,    0,   17,    0,   17],
       [2358,   56,   76, 1023, 1023,    0,   13,    0,   13],
       [2359,   57,   74, 1023, 1023,    0,   13,    0,   13],
       [2360,   54,   78, 1023, 1023,    0,   12,    0,   12]])

In [31]:
from BinConverter.core.Tint_Matlab import getpos

getpos(pos_file_from_bin, arena='BehaviorRoom')

(array([[-52.41932066],
        [-52.21751384],
        [-52.01570703],
        ...,
        [-19.72661634],
        [-19.72661634],
        [-19.72661634]]),
 array([[63.87185751],
        [63.46824388],
        [63.87185751],
        ...,
        [60.037528  ],
        [60.037528  ],
        [60.037528  ]]),
 array([[0.0000e+00],
        [2.0000e-02],
        [4.0000e-02],
        ...,
        [6.0094e+02],
        [6.0096e+02],
        [6.0098e+02]]),
 50.0)

In [19]:
def establish_mmap_to_pos_file(pos_file):
    """
    Generates a memory map (mmap) object connected to an Axona .pos file.

    Parameters:
    -------
    pos_file (Path or Str):
        Full filename of Axona file with any extension.

    Returns:
    -------
    mm (mmap or None):
        Memory map to .pos file position data
    """    
    fpos = open(pos_file, "rb")
    mmpos = mmap.mmap(
        fpos.fileno(),
        length=0,
        access=mmap.ACCESS_READ,
        offset=0
    )
    return mmpos

In [20]:
import mmap

pos_file = pos_file_from_bin

mmpos = establish_mmap_to_pos_file(pos_file)

bytes_packet = 20
print(pos_file)
footer_size = len('\r\ndata_end\r\n')
print(footer_size)
header_size = len(get_header_bstring(pos_file))
num_bytes = os.path.getsize(pos_file) - header_size - footer_size
num_packets = num_bytes // bytes_packet
print(num_packets)

pos = np.ndarray(
    shape=(num_packets,), 
    dtype=('>i2', 8),
    buffer=mmpos,
    offset=len(get_header_bstring(pos_file)) + 4,
    strides=bytes_packet
)
pos

/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data/06172021-HPC-B6-RAW/06172021-HPC-B6-RAW.pos
12
15050


array([[ 137,   42, 1023, ...,   17,    0,   17],
       [ 137,   44, 1023, ...,   10,    0,   10],
       [ 137,   42, 1023, ...,   13,    0,   13],
       ...,
       [ 129,   29, 1023, ...,   18,    0,   18],
       [ 129,   29, 1023, ...,   18,    0,   18],
       [ 129,   29, 1023, ...,   18,    0,   18]], dtype=int16)

In [21]:
np_pos

memmap([(150121, 137, 42, 1023, 1023, 0, 17, 0, 17),
        (150122, 137, 44, 1023, 1023, 0, 10, 0, 10),
        (150123, 137, 42, 1023, 1023, 0, 13, 0, 13), ...,
        (165161, 129, 29, 1023, 1023, 0, 18, 0, 18),
        (165161, 129, 29, 1023, 1023, 0, 18, 0, 18),
        (165161, 129, 29, 1023, 1023, 0, 18, 0, 18)],
       dtype=[('t', '>i4'), ('X', '>i2'), ('Y', '>i2'), ('x', '>i2'), ('y', '>i2'), ('PX', '>i2'), ('px', '>i2'), ('tot_px', '>i2'), ('unused', '>i2')])

Strangely, we get different data values after converting from .bin to .pos using the BinConverter tool. I did not think this was the case with the old data, let's try converting that instead:

In [43]:
from BinConverter.core.readBin import get_bin_data, get_raw_pos
from BinConverter.core.CreatePos import create_pos

# Full filename to .bin file
bin_filename = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/20201004_Raw.bin'

# Read bin file position data with BinConverter and show first 4 rows
raw_position = get_raw_pos(bin_filename)
print('position data read from `.bin` file with BinConverter tool:\n\n', raw_position[0:4, :].astype(int))


# Save to .pos file
pos_file_from_bin = bin_filename.replace('.bin', '.pos')
create_pos(pos_file_from_bin, raw_position)


# Read .pos file data and show first 4 rows
bytes_packet = 20
footer_size = len('\r\ndata_end\r\n')
header_size = len(get_header_bstring(pos_file_from_bin))
num_bytes = os.path.getsize(pos_file_from_bin) - header_size - footer_size
num_packets = num_bytes // bytes_packet

# Set dtypes
pos_dt = np.dtype([('t', ">i4"), ('X', ">i2"), ('Y', ">i2"), ('x', ">i2"), ('y', ">i2"), 
                   ('PX', ">i2"), ('px', ">i2"), ('tot_px', ">i2"), ('unused', ">i2")])

# Read position data from .pos file after conversion from .bin
np_pos = np.memmap(
    filename=pos_file_from_bin,
    dtype=pos_dt,
    mode='r',
    offset=len(get_header_bstring(pos_file_from_bin)),
    shape=(num_packets, ),
)
print('\n\n\nposition data read from `.pos` file with numpy (after converting from .bin with BinConverter):\n\n', np_pos[0:4,])


position data read from `.bin` file with BinConverter tool:

 [[2357   55   74 1023 1023    0   17    0   17]
 [2358   56   76 1023 1023    0   13    0   13]
 [2359   57   74 1023 1023    0   13    0   13]
 [2360   54   78 1023 1023    0   12    0   12]]



position data read from `.pos` file with numpy (after converting from .bin with BinConverter):

 [(2357, 55, 74, 1023, 1023, 0, 17, 0, 17)
 (2358, 56, 76, 1023, 1023, 0, 13, 0, 13)
 (2359, 57, 74, 1023, 1023, 0, 13, 0, 13)
 (2360, 54, 78, 1023, 1023, 0, 12, 0, 12)]


Finally, read `.pos` data that was not converted with the BinConverter tool. I believe this should now be formatted as specified in the file format manual: